In [0]:
import dlt
from datetime import datetime
from pyspark.sql.functions import lit
full_path = "/Volumes/dltcatalog/dltvolume/volume"

In [0]:
def dttime():
    # Get the current date and time
    current_datetime = datetime.now()

    formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_datetime

In [0]:
@dlt.table(name = 'bronze_employee')
def bronze_employee():
  df = spark.readStream.format("cloudFiles").option("cloudFiles.format","csv").option("header","true").option("sep",",").schema("ID int,Name string,DOB int,Gender string,Country string,Email string,ProcessedDate string").load(full_path)
  df = df.withColumn("ProcessedDate",lit(dttime()))
  #display(df)
  return df

In [0]:
dlt.create_streaming_table(name = 'silver_employee')

In [0]:
dlt.apply_changes(
  target = 'silver_employee',
  source = 'bronze_employee',
  keys = ["ID"],
  stored_as_scd_type= "1",
  sequence_by = "ProcessedDate"
)